In [19]:
import threading
from bs4 import BeautifulSoup
import requests
import json

In [20]:
# declare constants
URL = "https://www.carvolution.com/en/cars"
deductable = 1500
canton = "ZH"  # Zurich code
SOURCE = 'carvolution'
headers = {'Content-Type': 'application/json'}
CONFLICT_BRAND = {
    "vw": "Volkswagen",
    "mercedes": "Mercedes-Benz"
    }

In [21]:
def trim_model_name(model, brand):
    """
    Trims model names
    :param model: model name
    :param brand: brand name
    :return: clean model name
    """
    n = len(brand)
    model = model.strip(" .!")
    if brand.lower() == model[:n].lower():
        model = model[n:]
    
    elif brand in CONFLICT_BRAND:
        t = CONFLICT_BRAND[brand]
        n = len(t)
        if t == model[:n].lower():
            model = model[n:]
    elif brand in "Mercedes-Benz":
        t = "mercedes"
        n = len(t)
        if t == model[:n].lower():
            model = model[n:]
    elif brand in "Volkswagen":
        t = "vw"
        n = len(t)
        if t == model[:n].lower():
            model = model[n:]
    elif brand in "Skoda":
        t = "škoda"
        n = len(t)
        if t == model[:n].lower():
            model = model[n:]
    model = model.strip(" .!")
    model = model.strip(" .!")
    
    return model

In [33]:
def get_car_package_info(external_id, km, month, brand, model, car_url):
    """
    Get car package info
    :param external_id:
    :param km:
    :param month:
    :return:
    """
    # get car package info
    url = "https://api.carvolution.com/web/calculation/calculation-request"
    data = {
        "modelId": external_id,
        "subscriptionConfiguration": {
            "kmPerMonth": km,
            "durationInMonths": month,
            "retention": deductable,
            "canton": canton
            },
        "dateOfBirth": None,
        "additionalOptions": []
        }
    data = json.dumps(data)
    response = requests.post(url, data = data, headers = headers)
    if response.status_code == 200:
        print(response.content)
        return_data = json.loads(response.content)
        if return_data is not None:
            price = return_data["subscriptionAmountPerMonth"]
            if price is not None:
                
                json_data = {
                    "external_id": external_id,
                    "source": SOURCE,
                    "brand": brand,
                    "model": model,
                    "km": km,
                    "n_months": month,
                    "price": price,
                    "url": "https://www.carvolution.com" + car_url
                    }
                print(json_data)
                if price != '' and price is not None:
                    cars.append(json_data)

In [31]:
def get_each_car_detail(external_id, brand, model, car_url):
    """
    
    :param external_id:
    :param brand:
    :param model:
    :return:
    """
    global counter
    counter = counter + 1
    print("Counter - ", counter, " ", external_id)
    model = trim_model_name(model, brand)
    # get car package info
    all_threads = []
    for km in [850, 1200, 1850, 2500, 3250]:
        for month in [3, 6, 12, 18, 24, 36, 48]:
            thread = threading.Thread(
                target = get_car_package_info, args = (external_id, km, month, brand, model, car_url)
                )
            all_threads.append(thread)
            thread.start()
    for thread in all_threads:
        thread.join()

In [34]:
cars = []
counter = 0
response = requests.get(URL)
if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        section = soup.find("section", attrs = {"id": "cars"})
        all_cars_div = section.find_all(
            "div", attrs = {"class": "car-container col-12 col-md-6 col-lg-6 col-xxl-4 ng-star-inserted"}
            )
        for car_div in all_cars_div[:1]:
            car_url = car_div.find("a", attrs = {"class": "car ng-star-inserted"})['href']
            car = car_div.find("a", attrs = {"class": "car ng-star-inserted"})
            if car:
                images = car.find_all("img", attrs = {"class": "lazyload ng-star-inserted"})
                external_id = None
                for image in images:
                    img_src = image["data-src"]
                    if "md-" in img_src:
                        external_id = int(img_src.split("md-")[1].split("/")[0])
                        break
                if not external_id:
                    continue
                brand = car_div.find("h3", attrs = {"class": "brand"}).text
                if brand.lower() in CONFLICT_BRAND:
                    brand = CONFLICT_BRAND[brand.lower()]
                model = car_div.find("h2", attrs = {"class": "model"}).text
                get_each_car_detail(external_id, brand, model, car_url)

Counter -  1   330
b'{"modelId":330,"calculationDate":null,"financeCostId":null,"insuranceCostId":null,"youngDriverFeeCostId":null,"taxCostId":null,"tireCostId":null,"mixedPurposeCostId":null,"registrationCostId":null,"crossedOutPriceId":null,"subscriptionAmountPerMonth":null,"totalAmountPerMonth":null,"crossedOutPriceAmountPerMonth":null,"crossedOutPriceTotalAmountPerMonth":null,"crossedOutPriceAmountPerMonthDelta":null,"ageLimitForInsuranceCalculation":null,"youngDriverFee":null,"youngDriver":false,"initialFee":null,"homeDeliveryFee":null,"additionalKilometerPrice":null,"additionalOptions":[],"subscriptionConfigurationOptions":{"kmPerMonthOptions":[{"value":850,"delta":null,"crossedOutPriceDelta":null,"crossedOutPriceAmountPerMonth":null,"totalAmountPerMonth":null},{"value":1200,"delta":null,"crossedOutPriceDelta":null,"crossedOutPriceAmountPerMonth":null,"totalAmountPerMonth":null},{"value":1850,"delta":null,"crossedOutPriceDelta":null,"crossedOutPriceAmountPerMonth":null,"totalAmou

In [25]:
cars

[]